# Logging Examples for neural_analysis

This notebook demonstrates the centralized logging utilities provided by `neural_analysis.utils.logging`. These tools help make analyses reproducible, debuggable, and easier to track.

## Key Features:
- **configure_logging**: Set up logging level, format, file output
- **get_logger**: Get namespaced loggers per module
- **log_kv**: Structured key=value logging for metrics
- **log_section**: Visual section separators
- **log_calls**: Decorator for automatic function tracing

In [1]:
# Import logging utilities
import sys
import numpy as np
from neural_analysis.utils import (
    configure_logging,
    get_logger,
    log_kv,
    log_section,
    log_calls,
)

# Configure logging once at the start
# Level can be: DEBUG, INFO, WARNING, ERROR, CRITICAL
configure_logging(level="INFO")
print("✓ Logging configured at INFO level")

✓ Logging configured at INFO level


## Example 1: Basic logging with get_logger

Each module should get its own logger using `get_logger(__name__)` for proper namespacing.

In [2]:
# Get a namespaced logger
log = get_logger("example.basic")

# Log at different levels
log.debug("This debug message won't show (level=INFO)")
log.info("Loading dataset...")
log.warning("This is a warning")
log.error("This is an error (but doesn't raise)")

print("\n✓ Basic logging complete")

2025-11-06 11:53:55 | INFO | neural_analysis.example.basic | Loading dataset...


2025-11-06 11:53:55 | WARNING | neural_analysis.example.basic | This is a warning


2025-11-06 11:53:55 | ERROR | neural_analysis.example.basic | This is an error (but doesn't raise)



✓ Basic logging complete


## Example 2: Structured logging with log_kv

Use `log_kv` to log key-value pairs in a consistent format. Great for hyperparameters, metrics, or configuration snapshots.

In [3]:
# Log structured metrics
log_kv("config", {
    "dataset": "neural_recordings_01",
    "n_neurons": 120,
    "n_trials": 500,
    "sampling_rate": 30000,
})

# Log performance metrics
log_kv("metrics", {
    "accuracy": 0.934,
    "loss": 0.127,
    "f1_score": 0.891,
})

print("✓ Structured logging complete")

2025-11-06 11:53:55 | INFO | neural_analysis.kv | config: dataset='neural_recordings_01', n_neurons=120, n_trials=500, sampling_rate=30000


2025-11-06 11:53:55 | INFO | neural_analysis.kv | metrics: accuracy=0.934, loss=0.127, f1_score=0.891


✓ Structured logging complete


## Example 3: Visual sections with log_section

Use `log_section` to create clear separators between processing phases.

In [4]:
log_section("Phase 1: Data Loading")
log.info("Reading raw neural data")
log.info("Loaded 120 neurons × 500 trials")

log_section("Phase 2: Preprocessing")
log.info("Filtering signals")
log.info("Normalizing firing rates")

log_section("Phase 3: Analysis", char="-")
log.info("Computing distance metrics")
log.info("Clustering neurons by similarity")

print("\n✓ Section logging complete")

2025-11-06 11:53:55 | INFO | neural_analysis.section | ============================================================


2025-11-06 11:53:55 | INFO | neural_analysis.section |  Phase 1: Data Loading 


2025-11-06 11:53:55 | INFO | neural_analysis.section | ============================================================


2025-11-06 11:53:55 | INFO | neural_analysis.example.basic | Reading raw neural data


2025-11-06 11:53:55 | INFO | neural_analysis.example.basic | Loaded 120 neurons × 500 trials


...

2025-11-06 11:53:55 | INFO | neural_analysis.section |  Phase 3: Analysis 


2025-11-06 11:53:55 | INFO | neural_analysis.section | ------------------------------------------------------------


2025-11-06 11:53:55 | INFO | neural_analysis.example.basic | Computing distance metrics


2025-11-06 11:53:55 | INFO | neural_analysis.example.basic | Clustering neurons by similarity



✓ Section logging complete


## Example 4: Function tracing with @log_calls decorator

The `@log_calls` decorator automatically logs function entry/exit with timing information. Perfect for tracking which functions are called and how long they take.

In [5]:
import logging
import time

# Decorate functions to trace calls
@log_calls(level=logging.INFO, timeit=True)
def process_data(n_samples: int) -> np.ndarray:
    """Simulated data processing."""
    time.sleep(0.1)  # Simulate work
    return np.random.randn(n_samples, 10)

@log_calls(level=logging.INFO)
def compute_statistics(data: np.ndarray) -> dict:
    """Compute basic statistics."""
    return {
        "mean": float(data.mean()),
        "std": float(data.std()),
        "shape": data.shape,
    }

# Run traced functions
log_section("Running traced functions")
data = process_data(100)
stats = compute_statistics(data)

log_kv("results", stats)
print("\n✓ Function tracing complete")

2025-11-06 11:53:55 | INFO | neural_analysis.section | ============================================================


2025-11-06 11:53:55 | INFO | neural_analysis.section |  Running traced functions 


2025-11-06 11:53:55 | INFO | neural_analysis.section | ============================================================


2025-11-06 11:53:55 | INFO | neural_analysis.__main__ | → process_data(args=1, kwargs=0)


2025-11-06 11:53:55 | INFO | neural_analysis.__main__ | ← process_data completed in 100.12 ms


2025-11-06 11:53:55 | INFO | neural_analysis.__main__ | → compute_statistics(args=1, kwargs=0)


2025-11-06 11:53:55 | INFO | neural_analysis.__main__ | ← compute_statistics completed in 0.26 ms


2025-11-06 11:53:55 | INFO | neural_analysis.kv | results: mean=-0.015460802657241013, std=1.022322984736669, shape=(100, 10)



✓ Function tracing complete


## Example 5: File logging

You can write logs to a file by passing `file_path` to `configure_logging`. Logs will go to both console and file.

In [6]:
import tempfile
from pathlib import Path

# Create temp directory for log file
tmpdir = tempfile.TemporaryDirectory()
log_file = Path(tmpdir.name) / "analysis.log"

# Reconfigure to add file output
configure_logging(level="DEBUG", file_path=log_file)
log = get_logger("example.file")

log.debug("This debug message now appears (level=DEBUG)")
log.info("Performing analysis step 1")
log.info("Performing analysis step 2")
log_kv("timing", {"step1": 12.4, "step2": 8.7})

# Read back the log file
print(f"\n📄 Log file written to: {log_file}")
if log_file.exists():
    print("\nFile contents:")
    print("=" * 60)
    print(log_file.read_text())
    print("=" * 60)
else:
    print("⚠️  Log file not found (handler may buffer writes)")
    print("   In production, logs are typically flushed on close or periodically.")

2025-11-06 11:53:55 | INFO | neural_analysis.example.file | Performing analysis step 1


2025-11-06 11:53:55 | INFO | neural_analysis.example.file | Performing analysis step 2


2025-11-06 11:53:55 | INFO | neural_analysis.kv | timing: step1=12.4, step2=8.7



📄 Log file written to: /tmp/tmpe9isb9ap/analysis.log
⚠️  Log file not found (handler may buffer writes)
   In production, logs are typically flushed on close or periodically.


## Example 6: Environment variable control

You can set `NEURAL_ANALYSIS_LOG_LEVEL` environment variable to control the default log level without code changes.

```bash
export NEURAL_ANALYSIS_LOG_LEVEL=DEBUG
python your_script.py
```

This is useful for debugging in production without modifying code.

In [7]:
import os

# Demonstrate env var (normally set before running Python)
os.environ["NEURAL_ANALYSIS_LOG_LEVEL"] = "WARNING"

# This would now default to WARNING level
# configure_logging()  # Would use WARNING from env var

print("✓ Set NEURAL_ANALYSIS_LOG_LEVEL=WARNING")
print("  (In practice, set this in your shell before running Python)")

✓ Set NEURAL_ANALYSIS_LOG_LEVEL=WARNING
  (In practice, set this in your shell before running Python)


## Best Practices Summary

1. **Configure once** at application/notebook startup with `configure_logging()`
2. **Use get_logger(__name__)** in each module for proper namespacing
3. **Prefer structured logs** with `log_kv()` for metrics and config
4. **Add section markers** with `log_section()` for readability
5. **Trace expensive functions** with `@log_calls()` decorator
6. **Avoid print()** in library code; use logging instead
7. **Use appropriate levels**: DEBUG for details, INFO for progress, WARNING/ERROR for issues

See `docs/logging.md` for complete documentation and API reference.